### 4c_query_rewrite_and_retry.ipynb  
If the results are poor, asks LLaMA to rewrite or clarify the query.  
Repeats the vector search using the new version of the query.  
Loops until the matches are deemed acceptable or a retry limit is reached.

In [12]:
from langchain.chat_models import ChatOllama
from langchain.schema import SystemMessage, HumanMessage
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

set up embedder object and load the locally stored vector store that has the chunks from the og site content

In [13]:
embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.load_local("vectorstore/", embeddings=embedder, allow_dangerous_deserialization=True)

create llm object

In [14]:
llm = ChatOllama(model="llama3")

define some vars used later

In [15]:
original_query = "battery duration of hip and ankle configurations"
query = original_query
max_retries = 3
attempt = 0

In [16]:
while attempt < max_retries:
    
    # perform similarity search on the vector db - running same process as before 4a
    matches = vectorstore.similarity_search(query, k=3)
    context = "\n\n".join([m.page_content for m in matches])

    # ask llama if these results are relevant to the query - 4b process
    eval_messages = [
        SystemMessage(
            content="You are a helpful assistant that checks if a retrieved document answers the query."
        ),
        HumanMessage(
            content=(
                f"User query: {query}\n\n"
                f"Retrieved chunks:\n{context}\n\n"
                "Do these chunks fully or partially answer the query? "
                "Reply YES or NO with a short explanation."
            )
        )
    ]
    judgment = llm.invoke(eval_messages).content.strip()
    print(f"\nllama judgment: {judgment}")

    # if results are good, break out of the loop
    if judgment.upper().startswith("YES"):
        print(f"\nfinal query accepted after {attempt + 1} attempt(s): {query}")
        break

    # if results are poor, ask llama to rewrite the query
    rewrite_messages = [
        SystemMessage(
            content="You are an assistant that helps rewrite vague or unclear user queries to make them more effective for semantic document search."
        ),
        HumanMessage(
            content=(
                f"The query was: '{query}'\n\n"
                "The results were not relevant. Rewrite the query to be more precise or reworded to help find better results. "
                "Only return the new query text."
            )
        )
    ]
    new_query = llm.invoke(rewrite_messages).content.strip()
    print(f"\ntrying new query: {new_query}")

    # update query and attempt counter
    query = new_query
    attempt += 1


llama judgment: YES

The retrieved chunks provide partial answers to the query about battery duration of hip and ankle configurations. Specifically, they mention that the hip-and-ankle configuration operated for 35 minutes before reaching 50% of the voltage capacity of the battery. Additionally, it is mentioned that each configuration was evaluated in terms of its life span when operating on a certain battery, but the actual durations are not specified.

final query accepted after 1 attempt(s): battery duration of hip and ankle configurations


In [17]:
print(f"\nfinal matched chunks for query: '{query}'\n")
for i, m in enumerate(matches):
    print(f"chunk {i+1}:\n{m.page_content[:300]}...\n")


final matched chunks for query: 'battery duration of hip and ankle configurations'

chunk 1:
and hip-and-ankle configurations were able to operate for 35, 25, and 15 min, respectively, before reaching 50% of the voltage capacity of the battery (~900 mAh). Similarly, the elbow configuration...

chunk 2:
increase interest in this joint. We evaluated the life span of each configuration when operating on a 22.2-V, 1800-mAh LiPo battery. The hip, ankle, and hip-and-ankle configurations were able to...

chunk 3:
whereas the battery decreased until reaching the manufacturer’s recommended limit. When configured to provide simultaneous assistance to the hip and ankle joints, the device was able to operate for...

